# 数据源

## 相关技术

## 数据预处理

# 图谱构建

## 知识抽取

### 结构化内容中获取实体 

In [1]:
# -*- coding: utf-8 -*-
import re
file_name = 'D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点.txt'
out_file_name = 'D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点.csv'
dict_file_name = "D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点字典.txt"
pattern = r'[:|：]' 

# 得到当前层级数
def get_level(line):
    ret_level = -1
    lev = {
        1 : r'第[一|二|三|四|五|六]章',
        2 : r'[一|二|三|四|五|六|七|八|九]、',
        3 : r'（[一|二|三|四|五|六]）',
        4 : r'\d+、',
        5 : r'[⑴|⑵|⑶|⑷|⑸|⑹|⑺|⑻]'}
    for key in lev.keys():
        if re.match(lev[key],line):
            break
            
    if key:
        ret_level = key
    return ret_level, lev
details = []

with open(file_name,'r',encoding='utf-8') as f:
    current_lev_map = {}
    for line in f.readlines():
        line = line.strip('\n').strip()
        if len(line) > 0:
            result = re.split(pattern, line, maxsplit=1)
            level,lev_define = get_level(line)
            result = [x.strip('\n') for x in result]
#             print('level --> ',level, ' result --> ', result[0])
            result[0] = re.split(lev_define[level],result[0],maxsplit=1)[1]
            current_lev_map[level] = result[0]
            up_title = ''
            if level >= 2:
                up_title = current_lev_map[level-1]
            details.append([up_title.strip(), result[0].strip(), result[1].strip() if len(result) > 1 else '' ])
#             print('title:{}-->{}'.format(up_title, result))
import pandas as pd
import numpy as np
dataframe = pd.DataFrame(np.array(details),columns=['title','name','detail'])
dataframe.to_csv(out_file_name,index=False,sep=',',encoding='utf_8_sig')

with open(dict_file_name, 'w', encoding='utf-8') as f:
    content = dataframe.name.values.tolist()
    content.append('圆点')
    text = '\n'.join(content)
    f.writelines(text)
        
print('over !')

'''
清除所有数据
MATCH (n)
OPTIONAL MATCH (n)-[r]-()
DELETE n,r

注意生成的csv文件需保存到 neo4j 应用目录import 下
执行脚本：
LOAD CSV WITH HEADERS FROM "file:/math.csv" AS row
CREATE (n:Knowledge)
set n=row;
'''
    

over !


'\n注意生成的csv文件需保存到 neo4j 应用目录import 下\n执行脚本：\nLOAD CSV WITH HEADERS FROM "file:/math.csv" AS row\nCREATE (n:Knowledge)\nset n=row;\n'

### 非结构化数据抽取实体

## 关系抽取 

### 简单关系抽取（根据知识点文本内容抽取NEXT关系）

In [7]:

'''
1、根据文档结构建立PARTOF关系
执行脚本：
MATCH (a:Knowledge), (b:Knowledge) 
   WHERE a.name = b.title
CREATE (a)-[:INCLUDE]->(b) 
RETURN a,b 

2、根据CSV知识点内容中检索方式，建立USE关系
处理流程（极简流程）：
1） 对内容分词，然后通过分词去匹配， 注意（这里关系有可能是引用和使用两种，怎么样进行区分？？？）  分词工具包： https://github.com/lancopku/pkuseg-python
'''
import pandas as pd
import pkuseg
import numpy as np
from py2neo import Graph
dict_file_name = "D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点字典.txt"
file_name = 'D:\\PROJECT_TW\\twedu\\解决方案\\小学数学知识点.csv'
wordseg = pkuseg.pkuseg(user_dict=dict_file_name)
dataframe = pd.read_csv(file_name)
graph = Graph(uri='bolt://127.0.0.1:7687',auth=('neo4j','654321')) 

kn_dict = None
with open(dict_file_name,'r',encoding='utf-8') as f:
    lines = f.readlines()
    kn_dict = [x.strip() for x in lines]
kn_dict.remove('圆点')
# print(kn_dict)
# 检测前置情况, 检测 cut text 是否在包含filter name
def is_pre(cut_text,filter_name):
    pre_kn = []
    cut_text = np.array(cut_text)
    for item in kn_dict:
        result = np.where(cut_text==item)
        if len(result[0]) > 0 and item != filter_name:
            pre_kn.append(item)
    return pre_kn

# 检测两个知识点是否有关系
def has_rel(k1, k2):
    result = False
    rdata = graph.run("MATCH (K:Knowledge {name:'%s'}) - [r] -> (P:Knowledge {name:'%s'}) RETURN r" % (k1,k2)).data()
    if (len(rdata) > 0):
        result = True
    return result

# 创建知识点NEXT关系 ， k2 -> NEXT --> k1, K1需要先掌握K2知识点
def create_next_rel(k1,k2):
    graph.run("MATCH (K1:Knowledge {name:'%s'}), (K2:Knowledge {name:'%s'}) CREATE (K2) - [:NEXT] -> (K1) return K1,K2" % (k1,k2))

for idx, row in dataframe.iterrows():
    detail = row['detail']
    if isinstance(detail,str) :
        text = wordseg.cut(detail)
        using_words = is_pre(text, row['name'])
        using_words = set(using_words)
        using_rel = [x for x in using_words if not has_rel(x,row['name'])]
#         print('kn -> {} pre kn : {} cut txt : {}'.format(row['name'], is_pre(text, row['name']), text))
        
        for item in using_rel:
#             print('kn -> {} next rel kn -> {}'.format(row['name'], item))
            create_next_rel(row['name'], item)
    
        




### 根据知识点成绩关联建立关系 

In [33]:
# 1、https://blog.csdn.net/luanpeng825485697/article/details/79808669 生成样本数据
# 2、https://blog.csdn.net/weixin_43060843/article/details/90269688?depth_1-utm_source=distribute.pc_relevant.none-task&utm_source=distribute.pc_relevant.none-task
#    python faker 模拟数据
# 3、https://cloud.tencent.com/developer/article/1576219 Python轻松实现统计学中重要的相关性分析
# 4、https://blog.csdn.net/sinat_29957455/article/details/79007269 python pandas 相关性分析
# 5、https://aic-fe.bnu.edu.cn/docs/20181205101832069569.pdf An Automatic Knowledge Graph Construction System forK-12 
# 6、https://blog.csdn.net/Arwen_H/article/details/84938524 PCA 主成份分析
# 7、https://www.jianshu.com/p/5c1cefd11477 中位数，百分位数
# 8、https://blog.csdn.net/qq_36636519/article/details/86033415 Python之Pandas学习笔记(三) apply|applymap|map和grouppy
# 9、https://blog.csdn.net/CSDNBigBoy/article/details/97034126 python 利用scipy.stats生成截断正态分布

'''
生成知识点成绩模拟数据
知识点： 
模拟需检测知识点：平行四边形计算公式(K0)
相关知识点：整数加法(K1)，整数减法(K2) --> 整数乘法(K3) --> 整数除法(K4)、常用的面积单位(K5)-->单位之间的换算(K6)、
无关知识点：货币单位换算(K7)、质量常用换算(K8)、时间单位换算(K9)、鸡兔问题(K10)
判断标准 P(Si -->Sj) AND P(^Sj --> ^Si)  : 关系为Sj为Si的前题，  P(Si -->Sj) 掌握了Si就等于掌握了Sj,  P(^Sj --> ^Si) : 没有掌握Sj就没有掌握Si
'''
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import scipy.stats as stats

K_MAP = {'K0':'平行四边形计算公式', 'K1':'整数加法', 'K2':'整数减法', 'K3':'整数乘法', 'K4':'整数除法','K5':'常用的面积单位',
         'K6':'单位之间的换算','K7':'货币单位换算','K8':'质量常用换算','K9':'时间单位换算','K10':'鸡兔问题'}

# 生成范围之内的正太分布数据
def gen_k_performance(lower=60,uper=95, mu=80,sigma=10, size=1000):
    x = stats.truncnorm(int((lower-mu)/sigma), int((uper-mu)/sigma), mu, sigma)
    data = x.rvs(size)
    data = data.astype(np.int)
    return data

# 生成模拟成绩数据
def gen_all_data(size=1000):
    data = {}
    data['K1'] = gen_k_performance(size=size)
    data['K2'] = gen_k_performance(size=size)
    data['K3'] = ((data['K1'] + data['K2']) / 2).astype(np.int) + np.random.randint(-5,5,size)
    data['K4'] = ((data['K1'] + data['K2']) / 2).astype(np.int) + np.random.randint(-5,5,size)
    data['K5'] = gen_k_performance(size=size)
    data['K6'] = data['K5'] + np.random.randint(-5,5,size)
    data['K7'] = gen_k_performance(size=size)
    data['K8'] = gen_k_performance(size=size)
    data['K9']=  gen_k_performance(size=size)
    data['K10'] = gen_k_performance(size=size)
    data['K0'] = ((data['K3'] + data['K4'] + data['K6']) / 3).astype(np.int) +  np.random.randint(-5,5,size)
    return data


def coef_relation(k_data, km,kn):
    '''
    处理流程：
    1、先计算协方差， 抽取关系系数较大的
    2、两两分别计算影响度，找出关系
    '''    
    #    检测M是否是N的前置知识点
    #     判断标准 P(Si -->Sj) AND P(^Sj --> ^Si)  : 关系为Sj为Si的前题，  P(Si -->Sj) 掌握了Si就等于掌握了Sj,  P(^Sj --> ^Si) : 没有掌握Sj就没有掌握Si
    rel_forward_coef = len(np.where((k_data[kn]>=2) & (k_data[km] == 3) )[0]) / len(k_data) # P(Si -->Sj) 掌握了Si就等于掌握了Sj
    rel_backward_coef = len(np.where((k_data[km]==1) & (k_data[kn] == 1) )[0]) / len(k_data) #  P(^Sj --> ^Si) : 没有掌握Sj就没有掌握Si
    return rel_forward_coef, rel_backward_coef

# 返回 k_name的前置知识点
def get_relation(k_data,k_name,min_supp=0.05, min_coef=0.5):
    rel_map = {}
    for key in K_MAP.keys():
        if k_name != key:
            support, coef = coef_relation(k_data, key, k_name)
            if support >= min_supp and coef >= min_coef:
                rel_map['{}'.format(key)] = (support,coef)
    return rel_map

p_data  = gen_all_data(1000)
p_data_values = np.array([x.tolist() for x in p_data.values()])
p_data =  pd.DataFrame(p_data_values.T, columns=p_data.keys())
percen = np.percentile(p_data,[60, 80, 100])
pen_class = lambda x : 1 if x <= percen[0] else 2 if x > percen[0] and x <= percen[1] else 3  
pen_class_data = p_data.applymap(pen_class)

pen_class_data.to_csv('D:/PROJECT_TW/git/data/kg/k_performance.csv')



{'K1': (0.093, 0.518), 'K2': (0.085, 0.528), 'K3': (0.068, 0.613), 'K4': (0.078, 0.633), 'K5': (0.09, 0.503), 'K6': (0.101, 0.523)}


In [43]:
# 将前置知识点的权重更新到知识图谱
from py2neo import Graph
import numpy as np
import pandas as pd
K_MAP = {'K0':'平行四边形计算公式', 'K1':'整数加法', 'K2':'整数减法', 'K3':'整数乘法', 'K4':'整数除法','K5':'常用的面积单位',
         'K6':'单位之间的换算','K7':'货币单位换算','K8':'质量常用换算','K9':'时间单位换算','K10':'鸡兔问题'}
# match (k1:Knowledge)-[r:NEXT]->(k2:Knowledge) where r.weight>=0 return k1,r,k2 查询
# match (:Knowledge)-[r:NEXT]->(:Knowledge) where r.weight>=0 delete r 清除关系
pen_class_data = pd.read_csv('D:/PROJECT_TW/git/data/kg/k_performance.csv')
k_rel = {}
for ik in K_MAP.keys():
    # 合并MAP, 比较MAP，创建关系
    k_rel.update({ik: get_relation(pen_class_data, ik, min_supp=0.05, min_coef=0.5)})



graph = Graph(uri='bolt://127.0.0.1:7687',auth=('neo4j','654321')) 
# 检测两个知识点是否有关系
def has_next_rel(k1, k2):
    result = False
    rdata = graph.run("MATCH (K:Knowledge {name:'%s'}) - [r] -> (P:Knowledge {name:'%s'}) RETURN r" % (k1,k2)).data()
    if (len(rdata) > 0):
        result = True
    return result

# 创建知识点NEXT关系 ， k2 -> NEXT --> k1, K1需要先掌握K2知识点
def create_next_rel(k1,k2,weight):
    print('create next key {}->{} weight {}'.format(k1,k2,weight))
    graph.run("MATCH (K1:Knowledge {name:'%s'}), (K2:Knowledge {name:'%s'}) CREATE (K2) - [:NEXT {weight:%s}] -> (K1) return K1,K2" % (k1,k2,weight))

# 查询关系 MATCH (K1:Knowledge {name:'平行四边形计算公式'})-[relN:NEXT]- (K2:Knowledge {name:'整数加法'}) return K1,K2,relN
def query_next_rel(k1,k2):
    graph.run("MATCH (K1:Knowledge {name:'%s'}) -[relNext:NEXT]- (K2:Knowledge {name:'%s'})  return K1,K2" % (k1,k2,relNext))
    
for m_key in k_rel.keys():
    if len(k_rel[m_key].keys()) > 0:
        p_sum_value = sum(np.array(list(k_rel[m_key].values()))[:,0])
        n_sum_value = sum(np.array(list(k_rel[m_key].values()))[:,1])
#         print(m_key, ':', p_sum_value,':', n_sum_value)
        for p_key in k_rel[m_key].keys():
            #             print(K_MAP[p_key], '-->', K_MAP[m_key], ' has next rel :',has_next_rel(K_MAP[p_key],K_MAP[m_key]) )
            #    比较K相关性值
            p_value = k_rel[m_key][p_key][0]
            if m_key in k_rel[p_key] and p_value < k_rel[p_key][m_key][0]:
                continue
#             print(K_MAP[p_key], '-->', K_MAP[m_key], ' has next rel :',has_next_rel(K_MAP[p_key],K_MAP[m_key]))
            create_next_rel(K_MAP[m_key], K_MAP[p_key], p_value/p_sum_value) 
                
            
print(k_rel)



create next key 平行四边形计算公式->整数加法 weight 0.18058252427184465
create next key 平行四边形计算公式->整数减法 weight 0.1650485436893204
create next key 平行四边形计算公式->整数乘法 weight 0.13203883495145632
create next key 平行四边形计算公式->整数除法 weight 0.15145631067961166
create next key 平行四边形计算公式->常用的面积单位 weight 0.17475728155339804
create next key 平行四边形计算公式->单位之间的换算 weight 0.19611650485436893
create next key 整数乘法->整数加法 weight 0.3868421052631579
create next key 整数乘法->整数减法 weight 0.3394736842105263
create next key 整数乘法->整数除法 weight 0.27368421052631575
create next key 整数除法->整数加法 weight 0.3341404358353511
create next key 整数除法->整数减法 weight 0.31234866828087166
create next key 常用的面积单位->单位之间的换算 weight 0.8113207547169811
{'K0': {'K1': (0.093, 0.518), 'K2': (0.085, 0.528), 'K3': (0.068, 0.613), 'K4': (0.078, 0.633), 'K5': (0.09, 0.503), 'K6': (0.101, 0.523)}, 'K1': {'K3': (0.096, 0.51), 'K4': (0.097, 0.519)}, 'K2': {'K3': (0.105, 0.524), 'K4': (0.102, 0.533)}, 'K3': {'K1': (0.147, 0.51), 'K2': (0.129, 0.524), 'K4': (0.104, 0.601)},

array([0.146, 0.133, 0.101, 0.124])

### 从非结构化数据中提取关系、实体 

# 图谱扩展

## 增加资源节点 

In [ ]:
# 文档、PPT、视频之类 已结构化的数据， 无结构化的资源？？？？

## 增加习题节点 

In [ ]:
# 已结构化的数据， 无结构化的资源？？？？